# **이미지 패턴 인식**


---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---

- **(코랩에서)한글폰트 설치하기**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 런타입 > 세션 다시 시작

In [ ]:
import matplotlib.pyplot as plt

# 한글 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
# matplotlib에서 기본 폰트로 지정
plt.rcParams['font.family'] = 'NanumGothic'



---



- **이미지 패턴인식 방법**의 예:
    - 구조적 접근법
    - 통계적 접근법
    - 신경망적 접근법

- 이미지 특징(feature) 인식 방
    - 전통적(기하학·수학적 특징 기반)
    - 딥러닝(CNN 기반)

## **구조적 접근법** : 문법이나 규칙으로 정의

- **개념**
    - 이미지를 픽셀의 집합이 아닌, 더 단순한 기본 요소(Primitives)들의 조합으로 간주합니다. 그리고 이 요소들이 **어떻게 연결되고 배치되었는지**(구문, Syntax)를 '문법(Grammar)'이나 '규칙(Rule)'으로 정의하여 패턴을 인식하는 방식
- **동작 방식**
    1. 전처리 및 기본 요소 추출 (Preprocessing & Primitive Extraction): 기본적인 '형태' 정보만
        - 이때 가장 많이 사용되는 것이 바로 **'엣지**(Edge, 외곽선)
    2. 구조 기술 (Describe Relationships): 추출된 기본 요소들 간의 공간적 관계를 기술
        - 예: "선분 A는 선분 B와 '수직으로 만난다'", "곡선 C는 곡선 D의 '위에 있다'", "모서리 E는 두 선분의 '끝점에서 만난다'" 등
        - 이 관계들은 '그래프(Graph)'나 '트리(Tree)' 구조로 표현
    3. 구조적 매칭 (Structural Matching)
        - 미리 정의된 '모델(Model)' 또는 **'문법(Grammar)'**과 입력 이미지에서 추출한 구조를 비교
        - 찾던 모델의 문법과 일치하면, "패턴을 찾았다!"고 인식

- **예제: 고양이 구조 인식**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1. Colab 환경에서 이미지 불러오기
image_path = 'Cat03.jpg'
src_img_bgr = cv2.imread(image_path)

if src_img_bgr is None:
    print(f"오류: {image_path} 파일을 찾을 수 없거나 읽을 수 없습니다.")
    print("Colab 왼쪽에 Cat03.jpg 파일을 업로드했는지 확인해주세요.")
else:
    # 2. 구조적 접근법을 위한 전처리: 그레이스케일(Grayscale) 변환
    # Canny 엣지는 색상 정보가 아닌 '밝기(Intensity)' 변화를 기반으로 동작합니다.
    gray_img = cv2.cvtColor(src_img_bgr, cv2.COLOR_BGR2GRAY)

    # 3. (선택적) 노이즈 제거: 가우시안 블러 적용
    # 5x5 커널로 이미지를 부드럽게 만들어 작은 노이즈를 제거합니다.
    blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)

    # 4. Canny 엣지 검출 실행
    # cv2.Canny(image, minThreshold, maxThreshold)
    # minThreshold(최소 임계값): 100
    # maxThreshold(최대 임계값): 200
    # (이 값은 이미지마다 조절이 필요합니다)
    canny_edges = cv2.Canny(blurred_img, 100, 200)

    # 5. 결과 시각화 (원본, 그레이스케일, Canny 결과)
    plt.figure(figsize=(20, 7))

    # 원본 이미지 (RGB 변환)
    src_img_rgb = cv2.cvtColor(src_img_bgr, cv2.COLOR_BGR2RGB)
    plt.subplot(1, 3, 1)
    plt.imshow(src_img_rgb)
    plt.title("[1. 원본 이미지]")
    plt.axis('off')

    # 그레이스케일 이미지
    plt.subplot(1, 3, 2)
    plt.imshow(gray_img, cmap='gray')
    plt.title("[2. 그레이스케일 (엣지 검출 입력)]")
    plt.axis('off')

    # Canny 엣지 결과
    plt.subplot(1, 3, 3)
    plt.imshow(canny_edges, cmap='gray')
    plt.title("[3. Canny 엣지 검출 결과 (구조적 접근의 시작)]")
    plt.axis('off')

    plt.show()

## **통계적 접근접** **: 템플릿 매칭**
- **개념**
    - 원본 이미지(Source Image)에서, 우리가 찾고 싶은 견본(Template) 이미지 조각과 픽셀 단위로 가장 일치하는(유사도가 높은) 영역을 찾는 기법
- **작동원리**(통계적 이유)
    1. '템플릿' 이미지를 원본 이미지의 좌측 상단부터 픽셀 단위로 이동시킵니다 (이것을 '슬라이딩 윈도우'라고 부릅니다).
    2. 각 위치에서, 템플릿과 겹쳐지는 원본 이미지의 영역 간에 **유사도 점수**(통계량)를 계산합니다.
    3. 이 '유사도'를 계산하는 방식이 통계적입니다.
    4. 모든 위치를 탐색한 후, 유사도 점수가 가장 높은(또는 낮은) 위치를 "패턴을 찾은 위치"로 반환합니다.


- **예제: 고양이 눈 영역과 일치하는 패턴 매칭**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1. Colab 환경에 필요한 라이브러리 설치 확인 (보통 이미 설치되어 있습니다)
# !pip install opencv-python-headless matplotlib

# 2. 이미지 불러오기
# (Colab에 'Cat03.jpg' 파일이 업로드되어 있어야 합니다)
image_path = 'Cat03.jpg'
src_img_bgr = cv2.imread(image_path)

# 이미지를 못 불렀을 경우 예외 처리
if src_img_bgr is None:
    print(f"오류: {image_path} 파일을 찾을 수 없거나 읽을 수 없습니다.")
    print("Colab 왼쪽에 Cat03.jpg 파일을 업로드했는지 확인해주세요.")
else:
    # 3. 템플릿(Template) 생성: 원본 이미지에서 '눈' 부분 자르기
    # 원본 이미지를 matplotlib로 보기 위해 RGB로 변환
    src_img_rgb = cv2.cvtColor(src_img_bgr, cv2.COLOR_BGR2RGB)

    # 템플릿으로 사용할 영역을 NumPy 슬라이싱으로 자릅니다. (y:y+h, x:x+w)
    # (이 좌표는 Cat03.jpg 이미지에 맞춰 미리 찾아둔 값입니다)
    template_rgb = src_img_rgb[170:270, 335:435].copy()
    template_bgr = cv2.cvtColor(template_rgb, cv2.COLOR_RGB2BGR) # OpenCV용 BGR 템플릿

    # 4. 템플릿 매칭 실행
    # 템플릿 매칭은 BGR 또는 그레이스케일 이미지로 수행합니다.
    # TM_CCOEFF_NORMED: 정규화된 상관 계수 (1에 가까울수록 일치)
    result = cv2.matchTemplate(src_img_bgr, template_bgr, cv2.TM_CCOEFF_NORMED)

    # 5. 매칭 결과 분석
    # matchTemplate 결과(result)는 각 위치에서의 '유사도 맵'입니다.
    # minMaxLoc 함수로 최소값, 최대값, 그리고 해당 위치를 찾습니다.
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    print(f"찾은 패턴의 최고 유사도 점수: {max_val:.4f}")

    # TM_CCOEFF_NORMED를 사용했으므로, 최대값이 가장 일치하는 위치입니다.
    top_left = max_loc

    # 템플릿의 높이(h)와 너비(w)를 가져옵니다.
    h, w = template_bgr.shape[:2]
    bottom_right = (top_left[0] + w, top_left[1] + h)

    # 6. 결과 시각화
    # 원본 이미지(복사본)에 사각형 그리기
    result_img_rgb = src_img_rgb.copy()
    cv2.rectangle(result_img_rgb, top_left, bottom_right, (255, 0, 0), 3) # (빨간색, 3px 굵기)

    # Matplotlib로 Colab에 결과 표시
    plt.figure(figsize=(12, 7))

    plt.subplot(1, 2, 1)
    plt.imshow(template_rgb)
    plt.title("[Template] 우리가 찾으려는 패턴 (고양이 눈)")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(result_img_rgb)
    plt.title(f"[Result] 패턴 인식 결과 (유사도: {max_val:.2f})")
    plt.axis('off')

    plt.show()

## **신경망 접근법** : End-to-End Learning (종단 간 학습)

- **개념**
    - 인간의 뇌(신경망)가 시각 정보를 처리하는 방식을 모방한 "**End-to-End Learning**" (종단 간 학습)
    - <mark>**수백만 장의 데이터를 '스스로' 학습하여, 이미지 인식에 필요한 '특징(Feature)'과 '분류 규칙'을 동시에 찾아냄**</mark>
- **동작 방식**
    1. **특징 추출 (Feature Extraction)** - 자동화
        - CNN의 **합성곱 층**(Convolutional Layers)이 이 역할을 합니다.
        - (통계적 접근) 템플릿 매칭의 '필터(템플릿)'나 (구조적 접근) '엣지 검출기'를 사람이 설계하는 대신, CNN은 **수백만 개의 '학습 가능한 필터**'를 가집니다.
        - 이 필터들은 학습을 통해 이미지 인식에 유용한 특징을 자동으로 찾아냅니다.
    2. **계층적 특징 학습 (Hierarchical Features)**
        - 얕은 층(입력과 가까운 층)의 필터들은 '선', '모서리', '색상' 등 단순한 특징을 학습합니다. (마치 Canny 엣지처럼)
        - 깊은 층(출력과 가까운 층)의 필터들은 이 단순한 특징들을 조합하여 '눈', '코', '귀', '바퀴' 등 더 복잡하고 추상적인 '형태'나 '객체'를 학습합니다.
    3. **분류 (Classification)**
        - 마지막 **완전 연결 층**(**Fully Connected Layer**)은 추출된 복잡한 특징들(예: '눈'이 감지됨, '뾰족한 귀'가 감지됨)을 종합하여 최종 결론을 내립니다.
        - 예: "눈, 코, 뾰족한 귀 특징이 모두 존재하므로, 이 이미지는 98% 확률로 '고양이'다."
- <mark> [참고] "**눈을 인지하는 방법**"</mark>:
    - CNN은 '눈'이라는 템플릿(통계적)이나 '눈의 구조'(구조적)를 직접 찾는 게 아닙니다. 대신, 수많은 고양이 이미지를 학습하면서 '**고양이라고 판단하는 데 눈이 매우 중요한 단서임**'을 스스로 깨닫고, '눈'처럼 생긴 패턴에 강하게 반응하는 내부 필터를 자동으로 생성해 낸 것입니다.
    - **객체 탐지가 필요하다**.
    - **탐지** (Detection): 사진에 '무엇'이 '어디'에 있는지 경계 상자(Bounding Box)로 알려줌. (예: "고양이가 (x=100, y=120)부터 (x=480, y=450) 위치에 있다.")

In [ ]:
# =========================================
# [셀 1] 라이브러리 임포트 & 모델 준비
# =========================================
import torch
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

# 사전학습 가중치 & 전처리 파이프라인
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights).eval().to(device)
preprocess = weights.transforms()  # (Resize/CenterCrop/ToTensor/Normalize 포함)
categories = weights.meta["categories"]  # ImageNet 1k 클래스 이름


- **예제 : 고양이인지 판단**
    - ImageNet 사진들을 사전학습 한 ResNet-50으로 분류하고
    - Top-5 예측 중 ‘cat’이 포함되면 고양이로 판정한다.
    - (인터넷으로 가중치 자동 다운로드)

In [ ]:
# =========================================
# [셀 2] 이미지 업로드 → 추론 → "고양이 여부" 출력
# =========================================
from google.colab import files
import io

# # 업로드 창 표시 (여기서 이 대화의 고양이 이미지를 저장해 선택)
# uploaded = files.upload()
# if not uploaded:
#     raise RuntimeError("이미지를 업로드하세요.")

# img_name = list(uploaded.keys())[0]
# img = Image.open(io.BytesIO(uploaded[img_name])).convert("RGB")

# 파일명 직접 지정
image_path = "/content/Cat03.jpg"
img = Image.open(image_path).convert("RGB")

# 전처리 & 추론
with torch.no_grad():
    x = preprocess(img).unsqueeze(0).to(device)   # [1,3,H,W]
    logits = model(x)
    probs = torch.softmax(logits, dim=1)[0].cpu()

# Top-5 결과
top5_prob, top5_idx = torch.topk(probs, k=5)
top5_labels = [categories[i] for i in top5_idx.tolist()]

# "cat" 포함 여부로 간단 판정 (ImageNet의 고양이 관련 라벨: 'tabby, tiger cat, Persian cat, Siamese cat, Egyptian cat' 등)
is_cat = any("cat" in lbl.lower() for lbl in top5_labels)

print("=== Top-5 예측 ===")
for p, lbl in zip(top5_prob.tolist(), top5_labels):
    print(f"{lbl:25s} : {p*100:.2f}%")

print("\n판정:", "고양이 맞음 ✅" if is_cat else "고양이 아님/불확실 ❓")

# 이미지 보여주기
plt.imshow(img)
plt.axis("off")
plt.title("Uploaded image")
plt.show()


- **예제: 신경망이 '고양이' 인지하기**
    - 수 많은 고양이 사진으로 사전 학습된 vgg16 네트워크를 이용
    - 이미 훈련된 CNN이 '고양이'라고 판단할 때 이미지의 어느 부분을 '중요하게' 보는지(인지하는지) 시각화함
    - Heat Map 을 통해 CNN 이 어떻게 그 이미지를 특정 클라스로 예측했는지를 이해할 수 있다는 주장
    - Grad-CAM (Gradient-weighted Class Activation Mapping)
    - 참고 : https://wikidocs.net/135874

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import matplotlib.pyplot as plt

# 1. 이미지 불러오기 및 전처리
image_path = 'Cat03.jpg'
src_img_bgr = cv2.imread(image_path)

if src_img_bgr is None:
    print(f"오류: {image_path} 파일을 찾을 수 없거나 읽을 수 없습니다.")
    print("Colab 왼쪽에 Cat03.jpg 파일을 업로드했는지 확인해주세요.")
else:
    # VGG16 모델은 224x224 크기의 이미지를 입력으로 받음
    img = cv2.resize(src_img_bgr, (224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array) # VGG16에 맞는 방식으로 전처리

    # 2. 미리 훈련된 VGG16 모델 로드 (ImageNet 데이터로 학습됨)
    model = VGG16(weights='imagenet')

    # 3. 모델 예측
    preds = model.predict(img_array)
    top_pred = decode_predictions(preds, top=1)[0][0]
    print(f"VGG16 모델 예측: {top_pred[1]} (정확도: {top_pred[2]:.2%})")

    # 4. Grad-CAM 생성 (신경망이 '인지'하는 부분 시각화)

    # Grad-CAM을 계산할 모델 생성
    # VGG16의 마지막 합성곱 층('block5_conv3')의 출력과 모델의 최종 예측을 가져옴
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer('block5_conv3').output, model.output]
    )

    # 예측된 클래스(가장 확률 높은 클래스)의 인덱스
    pred_index = np.argmax(preds[0])

    # Gradient 계산
    with tf.GradientTape() as tape:
        last_conv_output, predictions = grad_model(img_array)
        class_output = predictions[:, pred_index]

    grads = tape.gradient(class_output, last_conv_output)

    # 가중치 계산 (Global Average Pooling)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # CAM 계산 (특징 맵 * 가중치)
    last_conv_output = last_conv_output[0]
    heatmap = last_conv_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # ReLU 적용 (양의 기여도만 확인)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = heatmap.numpy()

    # 5. 결과 시각화
    # 원본 이미지 크기로 히트맵 크기 조절
    heatmap = cv2.resize(heatmap, (src_img_bgr.shape[1], src_img_bgr.shape[0]))

    # 히트맵을 0~255 범위의 컬러맵으로 변환
    heatmap = np.uint8(255 * heatmap)
    heatmap_jet = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # 원본 이미지와 히트맵 합성
    superimposed_img = cv2.addWeighted(src_img_bgr, 0.6, heatmap_jet, 0.4, 0)

    # Matplotlib로 Colab에 결과 표시 (BGR -> RGB 변환)
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(src_img_bgr, cv2.COLOR_BGR2RGB))
    plt.title("[1. 원본 이미지]")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title(f"[2. 신경망의 '인지' 영역 (Grad-CAM)]\n'{top_pred[1]}'로 판단한 근거")
    plt.axis('off')

    plt.show()

## **예제 : 패턴인식 방법 동영상으로 확인하기**

- <mark>**배포된 8_data 폴더에 있는 코드를 안내에 따라 Visual Studio Code로 실행한다.**</mark>



---

